In [1]:
import os
import pickle
import argparse
import numpy as np
import pandas as pd
from tqdm import tqdm
from rdkit import Chem
from rdkit import DataStructs
from rdkit.Chem import AllChem
from collections import defaultdict
from sklearn.model_selection import StratifiedShuffleSplit

In [2]:
data = pd.read_csv("data/twosides_ge_500.zip")

In [3]:
data

,Drug1_ID,Drug1,Drug2_ID,Drug2,Y,New Y
0,CID000002173,CC1(C(N2C(S1)C(C2=O)NC(=O)C(C3=CC=CC=C3)N)C(=O...,CID000003345,CCC(=O)N(C1CCN(CC1)CCC2=CC=CC=C2)C3=CC=CC=C3,79,0
1,CID000002173,CC1(C(N2C(S1)C(C2=O)NC(=O)C(C3=CC=CC=C3)N)C(=O...,CID000003345,CCC(=O)N(C1CCN(CC1)CCC2=CC=CC=C2)C3=CC=CC=C3,85,1
2,CID000002173,CC1(C(N2C(S1)C(C2=O)NC(=O)C(C3=CC=CC=C3)N)C(=O...,CID000003345,CCC(=O)N(C1CCN(CC1)CCC2=CC=CC=C2)C3=CC=CC=C3,735,2
3,CID000002173,CC1(C(N2C(S1)C(C2=O)NC(=O)C(C3=CC=CC=C3)N)C(=O...,CID000003345,CCC(=O)N(C1CCN(CC1)CCC2=CC=CC=C2)C3=CC=CC=C3,959,3
4,CID000002173,CC1(C(N2C(S1)C(C2=O)NC(=O)C(C3=CC=CC=C3)N)C(=O...,CID000003345,CCC(=O)N(C1CCN(CC1)CCC2=CC=CC=C2)C3=CC=CC=C3,255,4
...,...,...,...,...,...,...
4576282,CID000003461,C1=CN(C(=O)N=C1N)C2C(C(C(O2)CO)O)(F)F,CID000003954,CN(C)C(=O)C(CC[NH+]1CCC(CC1)(C2=CC=C(C=C2)Cl)O...,1008,106
4576283,CID000003461,C1=CN(C(=O)N=C1N)C2C(C(C(O2)CO)O)(F)F,CID000003954,CN(C)C(=O)C(CC[NH+]1CCC(CC1)(C2=CC=C(C=C2)Cl)O...,769,313
4576284,CID000003461,C1=CN(C(=O)N=C1N)C2C(C(C(O2)CO)O)(F)F,CID000003954,CN(C)C(=O)C(CC[NH+]1CCC(CC1)(C2=CC=C(C=C2)Cl)O...,930,185
4576285,CID000003461,C1=CN(C(=O)N=C1N)C2C(C(C(O2)CO)O)(F)F,CID000003954,CN(C)C(=O)C(CC[NH+]1CCC(CC1)(C2=CC=C(C=C2)Cl)O...,72,48


In [4]:
drug_smile_dict = {}

In [7]:
for id1, id2, smiles1, smiles2, relation in zip(data["Drug1_ID"], data["Drug2_ID"], data["Drug1"], data["Drug2"], data["New Y"]):
    drug_smile_dict[id1] = smiles1
    drug_smile_dict[id2] = smiles2
# print(drug_smile_dict.values())
smiles = list(drug_smile_dict.values())

In [8]:
len(smiles)

645

In [9]:
drug_features_dict = {}

In [10]:
for i in range(len(smiles)):
    print(i)
    sim_feature = []
    # len(smiles)
    for j in range(len(smiles)):
        if smiles[i] != smiles[j]:
            m1 = Chem.MolFromSmiles(smiles[i])
            # print("m1")
            # print(m1)
            m2 = Chem.MolFromSmiles(smiles[j])
            # print("m2")
            # print(m2)
            fp1 = AllChem.GetMorganFingerprintAsBitVect(m1, 2, 1024)
            # print("fp1")
            # print(fp1)
            fp2 = AllChem.GetMorganFingerprintAsBitVect(m2, 2, 1024)
            # print("fp2")
            # print(fp2)
            sim = DataStructs.FingerprintSimilarity(fp1, fp2)
            # print("sim")
            # print(sim)
            sim_feature.append(sim)

    sim_feature = np.array(sim_feature)
    drug_features_dict[smiles[i]] = sim_feature

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [11]:
def save_data(data, filename):
    dirname = f'data/preprocessed/twosides'
    if not os.path.exists(dirname):
        os.makedirs(dirname)
    filename = dirname + '/' + filename
    with open(filename, 'wb') as f:
        pickle.dump(data, f)
    print(f'\nData saved as {filename}!')


In [12]:
 save_data(drug_features_dict, 'drug_fig_data.pkl')


Data saved as data/preprocessed/twosides/drug_fig_data.pkl!


In [11]:
def read_pickle(filename):
    with open(filename, 'rb') as f:
        obj = pickle.load(f)
    return obj

In [12]:
drug_graph = read_pickle(os.path.join('data/preprocessed/twosides/drug_fig_data.pkl'))

In [13]:
print(drug_graph["CC(C(O)=O)C1=CC2=C(C=C1)C1=C(N2)C=CC(Cl)=C1"])

[0.10169492 0.14       0.10638298 ... 0.0703125  0.07692308 0.12765957]


In [14]:
from test_representation_fig import get_drug_fig, get_drug_to_smiles

In [15]:
from drugbank import test_representation_fig

ds1 = test_representation_fig.get_drug_to_smiles()

dfig = get_drug_fig(ds1, "DB11630")

# embedding = torch.from_numpy(embedding)

print(dfig)


tensor([0.0645, 0.1058, 0.0192,  ..., 0.0866, 0.0364, 0.0385],
       dtype=torch.float64)
